In [1]:
import sys
#sys.path.append( '/home/greg/current/NMEG_utils/py_modules/' )
sys.path.append( '../NMEG_utils/py_modules/' )

#%matplotlib inline
import load_nmeg as ld
import transform_nmeg as tr
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
#import ipdb as ipdb

sns.set_style("white")

data_path = 'C:/Research_Flux_Towers/Flux_Tower_Data_by_Site/'
#data_path = '/home/greg/sftp/eddyflux/Fluxall_files/provisional/'

In [2]:
# Years to load
start = 2007
end = 2014
# Sites to load
site = 'GLand'
# Empty dict for hourly dataframes
hourly = dict()

# Fill dict with multiyear dataframes for each site in sites
def dparser( y, m, d, H, M, S ):
    yr = int( y )
    mon = int( m )
    day = int( d )
    hr = int( H )
    mn =  int( M )
    sec = int( S )
    
    # Some files have a line of zeros at the end - hack it
    #if yr != year_arg:
    #    yr = 1955
    return ( dt.datetime( yr, mon, day, hr, mn, sec ))

qc_dat = pd.DataFrame()
rbd_dat = pd.DataFrame();
for i in range( start, end+1) :
    path_qc = data_path + site + '/processed_soil/' + site + '_' + str( i ) + '_soilmet_qc.txt'
    path_rbd = data_path + site + '/processed_soil/' + site + '_' + str( i ) + '_soilmet_qc_rbd.txt'
    new_qc = pd.read_csv(path_qc, delimiter=',', parse_dates={'tstamp':[0, 1, 2, 3, 4, 5]},
                                 date_parser=dparser, index_col='tstamp')
    new_rbd = pd.read_csv(path_rbd, delimiter=',', parse_dates={'tstamp':[0, 1, 2, 3, 4, 5]},
                                 date_parser=dparser, index_col='tstamp')
    qc_dat = qc_dat.append( new_qc )
    rbd_dat = rbd_dat.append( new_rbd )
    
rbd_dat.index

DatetimeIndex(['2007-01-01 00:30:00', '2007-01-01 01:00:00',
               '2007-01-01 01:30:00', '2007-01-01 02:00:00',
               '2007-01-01 02:30:00', '2007-01-01 03:00:00',
               '2007-01-01 03:30:00', '2007-01-01 04:00:00',
               '2007-01-01 04:30:00', '2007-01-01 05:00:00', 
               ...
               '2014-12-31 19:30:00', '2014-12-31 20:00:00',
               '2014-12-31 20:30:00', '2014-12-31 21:00:00',
               '2014-12-31 21:30:00', '2014-12-31 22:00:00',
               '2014-12-31 22:30:00', '2014-12-31 23:00:00',
               '2014-12-31 23:30:00', '2015-01-01 00:00:00'],
              dtype='datetime64[ns]', name='tstamp', length=140256, freq=None, tz=None)

# get the soil columns

In [3]:
qc_cols = [s for s in qc_dat.columns if 'SWC' in s and 'tcor' not in s]
rbd_cols = [s for s in rbd_dat.columns if 'SWC' in s and 'tcor' not in s]
print(qc_cols)
print(rbd_cols)

['SWC_G1_12p5_AVG', 'SWC_G1_22p5_AVG', 'SWC_G1_2p5_AVG', 'SWC_G1_37p5_AVG', 'SWC_G1_52p5_AVG', 'SWC_G2_12p5_AVG', 'SWC_G2_22p5_AVG', 'SWC_G2_2p5_AVG', 'SWC_G2_37p5_AVG', 'SWC_G2_52p5_AVG', 'SWC_G3_12p5_AVG', 'SWC_G3_22p5_AVG', 'SWC_G3_2p5_AVG', 'SWC_G3_37p5_AVG', 'SWC_G3_52p5_AVG', 'SWC_O1_12p5_AVG', 'SWC_O1_22p5_AVG', 'SWC_O1_2p5_AVG', 'SWC_O1_37p5_AVG', 'SWC_O1_52p5_AVG', 'SWC_O2_12p5_AVG', 'SWC_O2_22p5_AVG', 'SWC_O2_2p5_AVG', 'SWC_O2_37p5_AVG', 'SWC_O2_52p5_AVG', 'SWC_O3_12p5_AVG', 'SWC_O3_22p5_AVG', 'SWC_O3_2p5_AVG', 'SWC_O3_37p5_AVG', 'SWC_O3_52p5_AVG', 'SWC_Unk1_Unk1_AVG', 'SWC_echo_G1_12p5_AVG', 'SWC_echo_G1_22p5_AVG', 'SWC_echo_G1_2p5_AVG', 'SWC_echo_G1_37p5_AVG', 'SWC_echo_G1_52p5_AVG', 'SWC_echo_G2_12p5_AVG', 'SWC_echo_G2_22p5_AVG', 'SWC_echo_G2_2p5_AVG', 'SWC_echo_G2_37p5_AVG', 'SWC_echo_G2_52p5_AVG', 'SWC_echo_O1_12p5_AVG', 'SWC_echo_O1_22p5_AVG', 'SWC_echo_O1_2p5_AVG', 'SWC_echo_O1_37p5_AVG', 'SWC_echo_O1_52p5_AVG', 'SWC_echo_O2_12p5_AVG', 'SWC_echo_O2_22p5_AVG', 'SWC_echo

In [4]:
#soilcols1 = [s for s in site_df.columns if 'SWC_echo_G1' in s]
selection = 'G3'
sel_cols = [s for s in qc_cols if selection in s]
qc_subset = qc_dat[sel_cols]
rbd_subset = rbd_dat[sel_cols]

xlims = [ dt.datetime( start, 1, 1 ), dt.datetime( end, 12, 31 )]
ylims = [0, 0.5]

fig1 = plt.figure(figsize=( 16.5, 7.5 ), dpi=150, facecolor='w', edgecolor='k')
for i, colname in enumerate( qc_subset.columns ):
    nsensors = len( qc_subset.columns )
    ax = fig1.add_subplot( nsensors, 1, i+1 )
    plt.plot( qc_subset.index, qc_subset[ colname ], '.r', ms=4 )
    plt.plot( rbd_subset.index, rbd_subset[ colname ], '.k', ms=4 )
    plt.ylim( ylims )
    plt.xlim( xlims )
    plt.text( dt.datetime( start, 2, 1 ), ylims[0]+.3, colname, size=14  )
    if i<nsensors-1:
        plt.setp( ax.get_xticklabels(), visible=False)
    
qc_subset.plot(figsize=( 16.5, 7.5 ))
#plt.ylim( ylims )
#plt.xlim( xlims )
plt.ylabel('x')


plt.show()